In [1]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

[WARNING] H2O requires colorama module of version 0.3.8 or newer. You have version 0.3.7.
You can upgrade to the newest version of the module running from the command line
    $ pip3 install --upgrade colorama


In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_161"; Java(TM) SE Runtime Environment (build 1.8.0_161-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.161-b12, mixed mode)
  Starting server from /opt/conda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpslst4djw
  JVM stdout: /tmp/tmpslst4djw/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpslst4djw/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,22 secs
H2O cluster version:,3.16.0.4
H2O cluster version age:,22 days
H2O cluster name:,H2O_from_python_unknownUser_ts6oxx
H2O cluster total nodes:,1
H2O cluster free memory:,25.14 Gb
H2O cluster total cores:,32
H2O cluster allowed cores:,32
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [3]:
df = h2o.import_file('../input/train.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
df_test = h2o.import_file('../input/test.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [17]:
df_test.head()

PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7,,S
894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,,Q
895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22,1,1,3.1013e+06,12.2875,,S
897,3,"Svensson, Mr. Johan Cervin",male,14,0,0,7538,9.225,,S
898,3,"Connolly, Miss. Kate",female,30,0,0,330972,7.6292,,Q
899,2,"Caldwell, Mr. Albert Francis",male,26,1,1,248738,29,,S
900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18,0,0,2657,7.2292,,C
901,3,"Davies, Mr. John Samuel",male,21,2,0,nan,24.15,,S


In [6]:
train, test = df.split_frame(ratios=[.7])

In [7]:
# Identify predictors and response
x = train.columns
y = "Survived"
x.remove(y)

In [8]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [9]:
# Run AutoML for 900 seconds
aml = H2OAutoML(max_runtime_secs = 900)
aml.train(x = x, y = y, training_frame = train, leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
# View the AutoML Leaderboard
aml.leaderboard
aml.leader

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid_0_AutoML_20180207_055702_model_3


ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.08196010543749273
RMSE: 0.28628675386313757
LogLoss: 0.2767090571019453
Mean Per-Class Error: 0.10699974422371894
AUC: 0.9464319208798704
Gini: 0.8928638417597408
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.20552518538232098: 


,0,1,Error,Rate
0,292.0,25.0,0.0789,(25.0/317.0)
1,25.0,160.0,0.1351,(25.0/185.0)
Total,317.0,185.0,0.0996,(50.0/502.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2055252,0.8648649,158.0
max f2,0.1569280,0.8835979,177.0
max f0point5,0.5474904,0.9122085,111.0
max accuracy,0.3674859,0.9043825,130.0
max precision,0.9999468,1.0,0.0
max recall,0.0164681,1.0,365.0
max specificity,0.9999468,1.0,0.0
max absolute_mcc,0.3674859,0.7944275,130.0
max min_per_class_accuracy,0.1779459,0.8864865,169.0
max mean_per_class_accuracy,0.2055252,0.8930003,158.0


Gains/Lift Table: Avg response rate: 36.85 %



...

Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2018-02-07 06:06:58,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2018-02-07 06:06:58,9 min 18.991 sec,50200 obs/sec,10.0,1,5020.0,0.3985145,0.5222211,0.8496803,2.7135135,0.1932271,0.3982738,0.5274121,0.8482981,2.4791667,0.1848739
,2018-02-07 06:07:03,9 min 23.474 sec,39096 obs/sec,350.0,35,175700.0,0.2862868,0.2767091,0.9464319,2.7135135,0.0996016,0.3477861,0.4178365,0.8884977,2.4791667,0.1428571


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
Fare,1.0,1.0,0.0091068
Cabin.E12,0.9069815,0.9069815,0.0082597
Ticket,0.8972361,0.8972361,0.0081710
Cabin.B50,0.8968905,0.8968905,0.0081678
Cabin.E24,0.8885797,0.8885797,0.0080921
---,---,---,---
Cabin.D45,0.5315903,0.5315903,0.0048411
Cabin.B57 B59 B63 B66,0.5239494,0.5239494,0.0047715
Cabin.C2,0.4719828,0.4719828,0.0042983
Embarked.missing(NA),0.0,0.0,0.0



See the whole table with table.as_data_frame()


In [11]:
preds = aml.leader.predict(df_test)

deeplearning prediction progress: |███████████████████████████████████████| 100%


/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'Cabin' has levels not trained on: [A11, A18, A21, A29, A9, B10, B11, B24, B26, B36, B45, B52 B54 B56, B61, C105, C116, C130, C132, C28, C31, C39, C51, C53, C55 C57, C6, C80, C89, C97, D22, D34, D38, D40, D43, E39 E41, E45, E52, E60, F, F E46, F E57]
  warnings.warn(w)


In [12]:
pr = preds['predict']
df_pr = pr.as_data_frame()

In [16]:
# Запись результата
result = df_test['PassengerId'].as_data_frame()
result.insert(1,'Survived', df_pr)
result.to_csv('Result.csv', index=False)